In [1]:
from importlib import reload
import audioDatastore.AudioDatastore as myads
reload(myads)
import os.path
from collections import Counter

import random
from python_speech_features import mfcc, delta
import scipy.io.wavfile as wav
from spafe.utils import vis
import matplotlib.pyplot as plt
import misc.vad as vad
from sklearn.mixture import GaussianMixture, _gaussian_mixture
reload(vad)
import numpy as np
from scipy.special import logsumexp
from collections import Counter
import helper_functions as hf
reload(hf)
import pickle

In [2]:
# todo:
# vad takes too much time, need to get a new one working

In [3]:
# create ads
# female ads
datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\SPEECH DATA\FEMALE\MIC"
female_ads = myads.AudioDatastore()
female_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)
Counter(female_ads.labels).values()
# male ads
datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\SPEECH DATA\MALE\MIC"
male_ads = myads.AudioDatastore()
male_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)
Counter(male_ads.labels).values()
# combine ads
ads = myads.AudioDatastore()
ads.set([datasetFolder], files=female_ads.files + male_ads.files, labels=female_ads.labels+male_ads.labels)

In [4]:
# split ads
developmentLabels = ["M01","M02","M03","M04","M06","M07","M08","M09","F01","F02","F03","F04","F06","F07","F08","F09"]
evaluationLabels = ["M05","M10","F05","F10"]
adsTrain = myads.subset(ads, developmentLabels)
adsEvaluate = myads.subset(ads, evaluationLabels)
numFilesPerSpeakerForEnrollment = 3
adsEnroll, adsDet = myads.split(adsEvaluate, 3)
adsTest, adsDet = myads.split(adsDet, 2)

In [5]:
speedUpExamples = True
if speedUpExamples:
    adsTrain, _ = myads.split(adsTrain, 30)
    adsDet, _ = myads.split(adsDet, 21)

In [6]:
# set sample rate,
fs, _ = wav.read(adsTrain.files[0])

In [7]:
reload = False

try:
    if reload :
        raise Exception('reloading')
    training_features = pickle.load(open('training_features.pickle','rb'))
except:
    zero_indices = []
    training_features = []
    for i in range(len(adsTrain.files)):
        if i%50 == 0:
            print('completed ' + str(i) + ' iterations')
        feat = hf.helper_feature_extraction(adsTrain.files[i])
        training_features.append(feat)
    print('completed training feature fe!')
    pickle.dump(training_features, open('training_features.pickle','wb'))

In [8]:
# remove [] vectors in training files and pop the file from ads
adsTrainCleaned, training_features_cleaned = myads.clean_zeros(adsTrain, training_features)

In [9]:
# collect normalization factors

import numpy as np
means = []
std = []
for feature in training_features_cleaned:
    if len(feature) > 0:
        means.append(np.mean(feature, axis=0))
        std.append(np.std(feature, axis=0))

means = np.array(means)
means = np.mean(means, axis=0)

std = np.array(std)
std = np.mean(std, axis=0)

class NormFactor:
  def __init__(self, m, s):
    self.means = m
    self.std = s

normFactors = NormFactor(means, std)

pickle.dump(normFactors, open('normFactors.pickle','wb'))

In [10]:
normalised_features = []

for i in range(len(training_features_cleaned)):
    normalised = (training_features_cleaned[i] - means) / std
    normalised = normalised - np.mean(normalised)
    normalised_features.append(normalised)

In [11]:
# train ubm
numComponents = 32
ubm = GaussianMixture(n_components=numComponents, covariance_type='diag')
train_features_flattened = np.array([item for sublist in normalised_features for item in sublist])
ubm.fit(train_features_flattened)
print('sklearn model ll = ' + str(ubm.score(normalised_features[0])))
pickle.dump(ubm, open('ubm.pickle','wb'))

sklearn model ll = -13.593252850195737


In [12]:
# load
ubm = pickle.load(open('ubm.pickle', 'rb'))
normFactors = pickle.load(open('normFactors.pickle','rb'))

In [13]:
for i in normalised_features:
    if len(i) == 0:
        print('sad face')

In [14]:
# BW stats

import gmmubm.helper_functions as hfgmm

# do stuff
numComponents = 32
numFeatures = normFactors.means.shape[0]

N = []
F = []

for feature in normalised_features:
    # BW stats, # TODO make this in to function
    n,f,s,l = hfgmm.helper_expectation(feature, ubm)
    N.append(np.reshape(n, (1,1,numComponents)))
    F.append(np.reshape(f, (numFeatures,1,numComponents)))

print('BW stats completed')

BW stats completed


In [15]:
# Expand stats into matrices and center
means = ubm.means_
muc = np.reshape(means, (numFeatures, 1, numComponents))
Nc = []
Fc = []
for i in range(len(training_features_cleaned)):
    Nc.append(np.repeat(np.reshape(N[i], (1, numComponents)), numFeatures))
    Fc.append(np.reshape(F[i] - np.multiply(N[i], muc), (-1, 1)))


In [16]:
Sigma = np.reshape(ubm.covariances_, (-1, 1))

In [36]:
# Training Total Variability Space

numTdim = 32
if speedUpExamples:
    numTdim = 16

T = np.random.random((Sigma.shape[0], numTdim))
T = T / np.linalg.norm(T)
I = np.eye(numTdim)
Ey = []
Eyy = []
Linv = []

numSpeakers = len(training_features_cleaned)

# for iterIdx = 1:numIterations

# Calculate the posterior distribution of the hidden variable
TtimesInverseSSdiag= (T / Sigma).T
for s in range(numSpeakers):
    L=(I + np.dot(np.multiply(TtimesInverseSSdiag,Nc[s]),T))
    Linv.append(np.linalg.pinv(L))
    Ey.append(np.dot(np.dot(Linv[s],TtimesInverseSSdiag),Fc[s]))
    Eyy.append(Linv[s] + np.dot(Ey[s],Ey[s].T))

# 2. Accumlate statistics across the speakers
Eymat=np.concatenate(Ey, axis=1)
FFmat=np.concatenate(Fc, axis=1)
Kt=np.dot(FFmat,Eymat.T)
K = np.hsplit(Kt.T, numComponents)

newT = []
for c in range(numComponents):
    AcLocal = np.zeros((numTdim, numTdim))
    for s in range(numSpeakers):
        AcLocal = AcLocal + N[s][:,:,c]*Eyy[s]

    # update T
    newTc = np.linalg.pinv(np.dot(AcLocal,K[c]))
    newT.append(newTc)

T = np.concatenate(newT, axis=0).shape
print('training total variability space')

In [37]:
np.concatenate(newT, axis=0).shape

(320, 16)

In [55]:
(np.concatenate(Ey, axis=1)).shape

(7568, 1)